In [4]:
#%pip install transformers --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
#%pip install torch --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-26 17:49:13.679851: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appr

In [8]:
def load_and_prepare(path):
    df = pd.read_csv(path, on_bad_lines='skip')
    df['label'] = df['Оценка'].apply(lambda x: 0 if x < 3 else (1 if x == 3 else 2))
    df = df.dropna(subset=['Отзыв','Оценка'])
    return df

In [9]:
df = load_and_prepare('yandex_reviews_main_last_right_version.csv')

In [10]:
df

,Название точки,Ссылка,Отзыв,Оценка,Тип организации,label
0,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Этот медицинский центр, лучшее, что мы видели ...",5.0,Детская поликлиника,2
1,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Очень приятные, вежливые и предусмотрительные ...",5.0,Детская поликлиника,2
2,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,"Редкий, крутейший и очень ответственный специа...",5.0,Детская поликлиника,2
3,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,Хочу выразить свою благодарность Болдыреву Вла...,5.0,Детская поликлиника,2
4,Медицинский центр доктора Болдырева,https://yandex.ru/maps/org/meditsinskiy_tsentr...,Уже более пяти лет посещаем врача Владимира Ге...,5.0,Детская поликлиника,2
...,...,...,...,...,...,...
106011,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Советую посещать данное заведение!,5.0,Госпиталь,2
106012,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,"Отличные врачи, новое оборудование. Спасибо за...",5.0,Госпиталь,2
106013,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Скоро прийдёт новый начальник и надеюсь постав...,3.0,Госпиталь,1
106014,ВС РФ платные медицинские услуги Главный военн...,https://yandex.ru/maps/org/vs_rf_platnyye_medi...,Отличный госпиталь.,5.0,Госпиталь,2


In [11]:
import re
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  
    text = re.sub(r'[^\w\s.,!?]', '', text)
    return text.strip()

df['Отзыв'] = df['Отзыв'].apply(clean_text)

In [8]:
train_df, temp_df = train_test_split(
    df, test_size=0.3, stratify=df['label'], random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.25, stratify=temp_df['label'], random_state=42
)


In [9]:
class_weights = compute_class_weight(
    'balanced', classes=np.unique(train_df['label']), y=train_df['label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [10]:
class_weights

tensor([2.0521, 6.8099, 0.4227])

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'DeepPavlov/xlm-roberta-large-en-ru'

tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer)
max_length = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/xlm-roberta-large-en-ru and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(35054, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out

In [12]:
import warnings
warnings.filterwarnings("ignore")

class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.encodings = tokenizer(
            texts.tolist(),
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        self.labels = torch.tensor(labels.tolist(), dtype=torch.long)

    def __len__(self): return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

In [13]:
train_ds = ReviewDataset(train_df['Отзыв'], train_df['label'], tokenizer, max_length)
val_ds   = ReviewDataset(val_df['Отзыв'], val_df['label'], tokenizer, max_length)
test_ds  = ReviewDataset(test_df['Отзыв'], test_df['label'], tokenizer, max_length)

In [14]:
def compute_loss_with_weights(self, model, inputs, return_outputs=False):
    labels = inputs.pop('labels')
    outputs = model(**inputs)
    logits = outputs.logits
    loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
    loss = loss_fct(logits, labels)
    return (loss, outputs) if return_outputs else loss

type(model).compute_loss = compute_loss_with_weights

In [15]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds  = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [16]:
training_args = TrainingArguments(
    output_dir='./out',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    #logging_dir='./logs',
    #logging_steps=100,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    learning_rate=3e-5,
    save_total_limit=2
)

In [17]:
#%pip install -U accelerate --upgrade

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()


 25%|██▌       | 585/2316 [16:56<50:08,  1.74s/it]   

  4%|▍         | 100/2316 [01:53<42:15,  1.14s/it]
                                               
  4%|▍         | 100/2316 [01:53<42:15,  1.14s/it]

{'loss': 0.2151, 'grad_norm': 2.2664871215820312, 'learning_rate': 2.871761658031088e-05, 'epoch': 0.17}



  9%|▊         | 200/2316 [03:46<39:40,  1.13s/it]
                                               
  9%|▊         | 200/2316 [03:46<39:40,  1.13s/it]

{'loss': 0.2029, 'grad_norm': 7.4995574951171875, 'learning_rate': 2.7422279792746114e-05, 'epoch': 0.35}



 13%|█▎        | 300/2316 [05:39<37:31,  1.12s/it]
                                               
 13%|█▎        | 300/2316 [05:40<37:31,  1.12s/it]

...

{'loss': 0.1038, 'grad_norm': 4.940734386444092, 'learning_rate': 1.5155440414507773e-06, 'epoch': 3.79}



 99%|█████████▉| 2300/2316 [1:00:50<00:17,  1.12s/it]
                                                  
 99%|█████████▉| 2300/2316 [1:00:50<00:17,  1.12s/it]

{'loss': 0.0981, 'grad_norm': 3.9979560375213623, 'learning_rate': 2.2020725388601036e-07, 'epoch': 3.97}



100%|██████████| 187/187 [01:09<00:00,  3.34it/s]/it]
                                                     


{'eval_loss': 0.23717711865901947, 'eval_accuracy': 0.926814939152329, 'eval_f1': 0.9272046228532559, 'eval_precision': 0.9276100728287517, 'eval_recall': 0.926814939152329, 'eval_runtime': 69.4158, 'eval_samples_per_second': 343.294, 'eval_steps_per_second': 2.694, 'epoch': 3.99}


In [ ]:
metrics = trainer.evaluate(test_ds)
print("Test metrics:", metrics)

In [13]:
from torch.utils.data import DataLoader, TensorDataset

In [14]:
texts = df['Отзыв'].tolist()
enc = tokenizer(texts, max_length=max_length, padding='max_length',
                truncation=True, return_tensors='pt')
dataset = TensorDataset(enc['input_ids'], enc['attention_mask'])

loader = DataLoader(dataset, batch_size=32)

model.eval()
all_probs = []
with torch.no_grad():
    for input_ids, attention_mask in loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
        probs = torch.nn.functional.softmax(logits, dim=1).cpu()
        all_probs.append(probs)

probs = torch.cat(all_probs, dim=0).numpy()

In [18]:
import os


output_dir = './model_save/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/tokenizer.json')

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


output_dir = './out/checkpoint-2316/'
model_name = 'DeepPavlov/xlm-roberta-large-en-ru'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = AutoModelForSequenceClassification.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

model.to(device)

max_length = 256



In [15]:
import pandas as pd
import numpy as np


probs_df = pd.DataFrame(
    probs,
    columns=['p_neg', 'p_neu', 'p_pos']
)
probs_df.index = df.index

probs_df['company'] = df['Название точки']  # или df['Компания'] и т.п.

# нормированная разность
probs_df['norm_diff'] = (probs_df['p_pos'] - probs_df['p_neg']) / (
    probs_df['p_pos'] + probs_df['p_neg'] + 1e-12
)
# взвешенный индекс
probs_df['weighted_score'] = (
    0 * probs_df['p_neg'] +
    0.5 * probs_df['p_neu'] +
    1 * probs_df['p_pos']
)
# доля положительных среди ненейтральных
probs_df['pos_over_non_neu'] = probs_df['p_pos'] / (
    probs_df['p_pos'] + probs_df['p_neg'] + 1e-12
)

agg = probs_df.groupby('company').agg(
    avg_p_positive        = ('p_pos', 'mean'),
    avg_norm_diff         = ('norm_diff', 'mean'),
    avg_weighted_score    = ('weighted_score', 'mean'),
    avg_pos_over_non_neu  = ('pos_over_non_neu', 'mean'),
    n_reviews             = ('p_pos', 'size')  # на всякий случай
).reset_index()

In [16]:
agg

,company,avg_p_positive,avg_norm_diff,avg_weighted_score,avg_pos_over_non_neu,n_reviews
0,3D Medica,0.923758,0.911661,0.942604,0.955830,26
1,9 Лечебно-диагностический центр Минобороны России,0.903530,0.816236,0.910737,0.908118,210
2,9 Лечебно-диагностический центр Минобороны Рос...,0.877539,0.802629,0.896992,0.901314,599
3,ABC-медицина,0.826561,0.683306,0.841866,0.841653,715
4,Advanced Fertility Clinic,0.731694,0.473385,0.743192,0.736692,37
...,...,...,...,...,...,...
273,Центральный военный клинический госпиталь имен...,0.929586,0.884713,0.940925,0.942357,183
274,Эксимер,0.747470,0.544877,0.773921,0.772438,506
275,ЭммаКлиник,0.673097,0.366378,0.696266,0.683189,48
276,Эстетическая клиника Век Адалин,0.914787,0.857542,0.925394,0.928771,155


In [17]:
agg.to_csv('companies_ethics_indexes_xlm_large.csv',index=False)